In [31]:
import json
import os
from pprint import pprint

In [32]:
def redif2jsonld(redif_file):
    ''' A function that gets a ReDIF input and converts it to a JSON-LD'''
    articles=[]
    article={}
    creators=[]
    creator={}
    for i in content:
        pair=i.split(':')
        j=i.replace(pair[0]+':','').strip()
        if pair[0].lower()=='template-type':
            if len(article)>0:
                article['creator']=creators
                creators=[]
                articles.append(article)
            article={};
            article['creator']=[]; article['keywords']=[]; 
            article['template']=i
        elif pair[0].lower()=='author-name':
            if len(creator)>0:
                creators.append(creator)
            creator={}
            creator['name']=j
        elif pair[0]=='Author-Email':
            creator['email']=j
        elif pair[0]=='Author-Workplace-Name':
            creator['affiliation']=j
        elif pair[0].lower()=='title':article['title']=j
        elif pair[0].lower()=='year':article['date']=j
        elif pair[0].lower()=='pages':article['pages']=j
        elif pair[0].lower()=='volume':article['volume']=j
        elif pair[0].lower()=='issue':article['issue']=j
        elif pair[0].lower()=='file-url':article['url']=j
        elif pair[0].lower()=='abstract':article['abstract']=j
        elif pair[0].lower()=='keywords':article['keywords']=[keyword.strip() for keyword in j.split(',')]
    jd=json.dumps(articles,ensure_ascii=False)
    jd = unicode(jd, 'utf-8',errors='ignore')
    jl=json.loads(jd,encoding='utf-8')
    return (jl)

In [ ]:
#Run: put any number of ReDIF files in the current folder 
#Example: isre-0101-1704-BOM.rdf
data='../../../data/'
rdf_folder=data+'rdf/'
json_folder=data+'json/'
i=0
for root, dirs, files in os.walk(rdf_folder):
    path = root.replace(rdf_folder,'').split(os.sep)
    if len(path)==2:
        i=i+1
        if (i % 1000)==0: print i,
        jsonld_file=json_folder+root.replace('../../../data/rdf/','').replace('/','_')+'.json'
        objects={}
        objects['items']=[]
        for file in files:        
            if file[-4:]=='.rdf' or file[-6:]=='.redif':
                with open(root+'/'+file) as f:
                    content = f.readlines()
                # you may also want to remove whitespace characters like `\n` at the end of each line
                content = [x.strip() for x in content] 
                json_data=redif2jsonld(file)        
            objects['items']=objects['items']+json_data
        with open(jsonld_file, 'a') as outfile:
            json.dump(objects, outfile)

In [91]:
with open(data+'mongo_json.json','a') as mongo_json:
    for json_file in os.listdir(data+'json'):
        with open(data+'json/'+json_file) as json_str:
            json_object=json.load(json_str)
        for item in json_object['items']:
            mongo_json.write(json.dumps(item)+'\n')